In [100]:
import numpy as np
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
from uszipcode import ZipcodeSearchEngine
from geopy.geocoders import Nominatim

import pandas as pd
import seaborn.apionly as sns
from datetime import date, datetime
from haversine import haversine

# statistics package
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
from random import randint
# packages for mapping
from mpl_toolkits.basemap import Basemap

# packages for interactive graphs
from ipywidgets import widgets, interact
from IPython.display import display
from copy import deepcopy as copy
import time
from geopy.geocoders import Nominatim

from time import sleep
import requests
import time,datetime
%matplotlib inline

In [101]:
findzip = ZipcodeSearchEngine()

In [119]:
historical_data = pd.read_csv('clean_data_2.csv')

In [120]:
historical_data.head()

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,...,pickup_date,dropoff_date,distance,speed,day_interval,waiting_time,waiting_penalty,pickup_zipcode,dropoff_zipcode,weather
0,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-14 19:53:00,2013-01-14 20:12:00,1,1140,10.65,...,2013-01-14,2013-01-14,17.309398,54.661255,2,0.000000,0.000000,11371,11205,Rain
1,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-14 20:28:00,2013-01-14 20:35:00,1,420,1.78,...,2013-01-14,2013-01-14,3.397114,29.118117,2,349.630322,0.485598,10278,10010,Rain
2,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-14 20:40:00,2013-01-14 20:45:00,1,300,1.35,...,2013-01-14,2013-01-14,1.215656,14.587874,2,63.033845,0.087547,10003,10014,Rain
3,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-14 20:50:00,2013-01-14 20:58:00,1,480,1.99,...,2013-01-14,2013-01-14,2.784876,20.886573,2,1261.056819,1.751468,10012,10009,Rain
4,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-14 21:04:00,2013-01-14 21:11:00,1,420,1.33,...,2013-01-14,2013-01-14,1.680562,14.404815,2,1878.058306,2.608414,10009,10012,Rain


In [121]:
historical_data.keys()

Index(['medallion', 'hack_license', 'vendor_id', 'rate_code',
       'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'trip_time_in_secs', 'trip_distance',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'fare_amount', 'payment_type', 'surcharge',
       'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount', 'pickup_day',
       'pickup_weekday', 'pickup_hour', 'pickup_minute', 'pickup_time',
       'dropoff_hour', 'pickup_date', 'dropoff_date', 'distance', 'speed',
       'day_interval', 'waiting_time', 'waiting_penalty', 'pickup_zipcode',
       'dropoff_zipcode', 'weather'],
      dtype='object')

## Best Revenue from Historical Data

In [122]:
best_revenue = historical_data.groupby(['hack_license','pickup_date'])['total_amount'].sum()

In [123]:
np.max(best_revenue.reset_index()['total_amount'])

650.1

## Preprocess

In [124]:
historical_data["dropoff_minute"] = pd.to_datetime(historical_data["dropoff_datetime"]).apply(lambda x: x.minute)
historical_data["dropoff_time"] =historical_data["dropoff_hour"] + (historical_data["dropoff_minute"] / 60)

## Reinforcement Model

In [125]:
class TaxiWorld():
    def __init__(self):
        self.waiting_penalty = 5.00
        pass
                
    def getReward(self, row):
        if row['waiting_time']/3600<=2:
            return row['total_amount'] - row['waiting_time']/3600*self.waiting_penalty
        else:
            return row['total_amount']
        
    
    def set_waiting_penalty(self, penalty):
        self.waiting_penalty = penalty
        
    def getState(self):
        print('Current State')
        return self.s
    
    def setState(self, s):
        self.s = s
        
    def getStateSize(self, trip_and_fare):
        return len(trip_and_fare['zip_code'].unique())
    
    def getZipCodes(self, trip_and_fare):
        return self.zip_codes
    
    def getActionSize(self):
        return len(self.actions)

    def nextAction(self, s):
        print('Calculate Next Action based on state')
        
    def check_Q_table(self, s):
        if s not in self.Q:
            self.Q[s] = dict((action, 0.0) for action in self.actions)
            

In [126]:
env = TaxiWorld()

In [127]:
class TaxiRevenue:
    def __init__(self, env, penalty=5.0):
        self.env = env
        self.env.set_waiting_penalty(penalty)
        self.Q = dict()
        self.dropPickMap= dict()
        
    def load_Q(self, _Q):
        self.Q = _Q
        
    def generate_key(self, row):
        return str(row['pickup_zipcode']) + '_'  + str(row['pickup_weekday']) + '_' + str(row['day_interval']) + '_' + str(row['weather'])
    
    def get_tuples(self, key):
        tuples = key.split('_')
        return {'pickup_zipcode': tuples[0],
                   'pickup_weekday': tuples[1],
                   'day_interval': tuples[2],
                   'weather': tuples[3]}
    
    def get_meantriptime(self, data):
        map=historical_data.groupby(['pickup_zipcode', 'dropoff_zipcode'])['trip_time_in_secs'].mean()
        return map.reset_index()
    
    def isValidPickup(self, taxidata, dropPickMap):
        key=self.generate_key(taxidata)
        if key in dropPickMap:
            return True
        else:
            return False
        
    def generate_dropPickMap(self, data):
        for index,row in data.T.iteritems():
            key = self.generate_key(row)
            if key not in self.dropPickMap:
                self.dropPickMap[key] = []
            if str(row['dropoff_zipcode']) not in self.dropPickMap[key]:
                self.dropPickMap[key].append(str(row['dropoff_zipcode']))
        return self.dropPickMap
    
    def getDropoff(self, key, dropPickMap):
        posDropoff= dropPickMap[key]
        print("dropoffs",posDropoff )
        rand=randint(0, len(posDropoff)-1)
        return posDropoff[rand]
         
    def getDayInterval(self,hour):
        if hour>=6 and hour<12:
            return 0
        elif hour>=12 and hour<18:
            return 1
        elif hour>=18 and hour<24:
            return 2
        else:
            return 3
        
    def getAction(self, taxidata, _Q,dropPickMap, const):
        best_reward=0
        #best_key = 0
        s = self.generate_key(taxidata)
        best_key=s
        #print(taxidata)
        if s in _Q:
            best_reward=_Q[s][0]*_Q[s][1]/const
        
        zipcode=findzip.by_zipcode(taxidata['dropoff_zipcode'])
        temp=taxidata
        possible_pickup=[]
        returns=100
        while not possible_pickup:
            pickup_res = findzip.by_coordinate(zipcode['Latitude'], zipcode['Longitude'], radius=10, returns=returns)
            for i in range(len(pickup_res)):
                temp['pickup_zipcode']=pickup_res[i]['Zipcode']
                #print(temp['pickup_zipcode'])
                if self.isValidPickup(temp, dropPickMap):
                    possible_pickup.append(pickup_res[i]['Zipcode'])

            print("valid pickup", possible_pickup)
            returns+=10
        
        for key in _Q:
            tuples=self.get_tuples(key)
               
            
            if tuples['pickup_zipcode'] in possible_pickup and tuples['weather']==str(taxidata['weather']) and tuples['pickup_weekday']==str(taxidata['pickup_weekday']) and tuples['day_interval']==str(taxidata['day_interval']):
            
                tempReward=_Q[key][0]*_Q[key][1]/const
                
                if tempReward>best_reward:
                    #print("in temp reward: ",tempReward)
                    #print("in best reward: ",best_reward)
                    best_key=key
                    best_reward= tempReward
                    #print(self.get_tuples(best_key))
                    
        return best_key
    
    def build_Q_table(self, data):
        for index,row in data.T.iteritems():
            key = self.generate_key(row)
            reward = self.env.getReward(row)
            if key not in self.Q:
                self.Q[key] = [reward,1]
            else:
                self.Q[key] = [self.Q[key][0] +reward, self.Q[key][1] + 1]
                
    def greedy(self, s):
        return np.argmax(self.Q[s[0]]) 

    def epsilon_greed(self, epsilon, s):
        if np.random.rand() < epsilon:
            return np.random.randint(self.n_a)
        else:
            return self.greedy(s)
        
    def train(self, trip_and_fare):
        self.build_Q_table(trip_and_fare)
        return self.Q
        
    def distance(self, lat1, lon1, lat2, lon2):
        """calculates the Manhattan distance between 2 points
            using their coordinates

        Parameters
        ----------
        lat1: float
            latitude of first point

        lon1: float
            longitude of first point

        lat2: float
            latitude of second point

        lon2: float
            longitude of second point

        Returns
        -------
        d: float
            The Manhattan distance between the two points in kilometers

        """

        d = haversine((lat1, lon1), (lat2, lon1)) + haversine((lat2, lon1), (lat2, lon2))
        return d
    
    def find_distance(self, zip1, zip2):
        zipcode1=findzip.by_zipcode(zip1)
        zipcode2=findzip.by_zipcode(zip2)
        dis=self.distance(zipcode1['Latitude'], zipcode1['Longitude'],zipcode2['Latitude'], zipcode2['Longitude'])
        return dis
    
    
    def test(self, taxidata, _Q, dropPickMap,meanTrip, maxstep=1000):
        # TODO: Add your test routine that exmines learned policy
        #      Using the learned policy, generate the trajectory of 
        curr_dropoff_hour=taxidata['dropoff_time']
        #curr_pickup=taxidata['dropoff_time']
        end_time=curr_dropoff_hour+ 12
        
        trace = []
        rewards = []
        
        while curr_dropoff_hour <= end_time:
            
            best_key=self.getAction( taxidata, _Q,dropPickMap, 1000)
            pickup_tuple= self.get_tuples(best_key)
            
            if int(pickup_tuple['pickup_zipcode'])!=taxidata['dropoff_zipcode']:
                dis=self.find_distance(taxidata['dropoff_zipcode'], int(pickup_tuple['pickup_zipcode']))
                curr_dropoff_hour+= dis/taxidata['speed']
                print("roaming time", dis/taxidata['speed'] )
                
            dropoff=self.getDropoff(best_key, dropPickMap)
                
            print("going from ",pickup_tuple['pickup_zipcode'], " to ",  dropoff, " at ", curr_dropoff_hour)
            #curr_dropoff=curr_dropoff+meanTrip[meanTrip['pickup_zipcode']]
            #print(pickup_tuple['pickup_zipcode'])
            temptrip = meanTrip[(meanTrip['pickup_zipcode']==int(pickup_tuple['pickup_zipcode'])) & (meanTrip['dropoff_zipcode']==int(dropoff))]
            meanTripTime= temptrip.iloc[0]['trip_time_in_secs']
            #print(meanTripTime)
            
            curr_dropoff_hour=curr_dropoff_hour+meanTripTime/3600
            taxidata['dropoff_zipcode']=int(dropoff)
            taxidata['day_interval']=self.getDayInterval(curr_dropoff_hour)

## Train the Model

In [128]:
taxiRevenue = TaxiRevenue(env)

In [129]:
with open('saved_Q.json', 'r') as fp:
    saved_Q = json.load(fp)

In [130]:
taxiRevenue.load_Q(saved_Q)

In [131]:
_Q = taxiRevenue.train(historical_data)

In [132]:
_Q

{'11371_6_2_Rain': [38212.62251191671, 1074],
 '10023_6_2_Rain': [19556.737238821737, 1769],
 '10016_6_2_Rain': [19961.29859102639, 1790],
 '10010_6_2_Rain': [24081.905190834, 2121],
 '10003_6_2_Rain': [35698.74148506436, 3152],
 '11201_6_2_Rain': [5317.122926698532, 331],
 '10028_6_2_Rain': [12963.753837204888, 1166],
 '10119_0_3_Rain': [8370.705588819254, 685],
 '10154_0_3_Rain': [1403.172274101166, 106],
 '10009_6_3_Rain': [6711.153483366587, 433],
 '10119_6_3_Rain': [3509.9984912623954, 248],
 '10029_6_0_Rain': [3091.4284586309354, 272],
 '10020_6_0_Rain': [14960.110978809993, 1316],
 '10021_6_2_Rain': [10686.53295019338, 1015],
 '10075_6_2_Rain': [10406.605196643264, 949],
 '10011_6_2_Rain': [9293.630552894441, 784],
 '10023_0_3_Rain': [3197.2910187956127, 211],
 '11371_0_3_Rain': [12577.819989311882, 395],
 '11369_0_3_Rain': [4483.573823660346, 130],
 '10165_0_3_Rain': [607.7979866996287, 60],
 '10029_0_3_Rain': [1113.553543819454, 73],
 '10024_0_3_Rain': [3742.244797688736, 240]

In [133]:
with open('saved_Q.json', 'w') as fp:
    json.dump(_Q, fp)

## Generate Test Data

In [134]:
dropPickMap=taxiRevenue.generate_dropPickMap(historical_data)
meanTrip= taxiRevenue.get_meantriptime(historical_data)

## Test the Model

In [135]:
taxiRevenue.test(historical_data.iloc[14000], _Q, dropPickMap, meanTrip)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


valid pickup ['10014', '10011', '10012', '10013', '10003', '10199', '10119', '10001', '10282', '10010', '10278', '10018', '10007', '10009', '10016', '10036', '10006', '10165', '10168', '10173', '10280', '10002', '10170', '10038', '10103', '10174', '10020', '10167', '10112', '10004', '10111', '10017', '10019', '10154', '10005', '10022', '10069', '11109', '11201', '10065', '11222', '10023', '10044', '11211', '10021', '11101', '10075', '10024', '11205', '10028', '11231', '11217', '11206', '10128', '11106', '11238', '11104', '10025', '11102', '11215', '11216', '10029', '11378', '11221', '11103', '10026', '11377', '11232', '10115', '11225', '10035', '10027']
roaming time 0.3557921956221921
dropoffs ['10111', '10012', '10020', '10022', '10018', '10011', '10010', '10173', '10065', '10278', '10075', '10001', '10119', '10282', '10199', '10112', '10168', '10016', '11369', '10014', '10007', '10165', '10154', '11232', '10017', '10167', '7114', '10021', '10028', '10128', '10003', '10019', '10038', 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


valid pickup ['10075', '10021', '10028', '10065', '10128', '10044', '10022', '10023', '10024', '10154', '10029', '11102', '10111', '10112', '10167', '10017', '10020', '10170', '10174', '11106', '10173', '10168', '10165', '10019', '11109', '10069', '10025', '10035', '11101', '10026', '10036', '10016', '10018', '11103', '10119', '10199', '11105', '10001', '10115', '11104', '10010', '10027', '10454', '10037', '11222', '10030', '10003', '10011', '10009', '11370', '11377', '10451', '10031', '10014', '10455', '10012', '10039', '11372', '11378', '11371', '11211', '10002', '10474', '10013', '11369', '11373', '10103', '10278', '10032', '11206', '10452', '10007', '10282', '10459', '10038', '10006', '11201']
roaming time 0.6971822449308642
dropoffs ['10111', '10012', '10020', '10022', '10018', '10011', '10010', '10173', '10065', '10278', '10075', '10001', '10119', '10282', '10199', '10112', '10168', '10016', '11369', '10014', '10007', '10165', '10154', '11232', '10017', '10167', '7114', '10021', 